In [5]:
import sys
import os
import torch
import pickle

# Agrega la carpeta raíz del proyecto al sys.path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

from ibioml.models import MLPModel
from ibioml.tuner import run_study

path = os.path.join(os.getcwd(), 'data')

# Cargar datos
with open(os.path.join(path, 'bins200ms_preprocessed_withCtxt.pickle'), 'rb') as f:
    X, y, T = pickle.load(f)

with open(os.path.join(path, 'bins200ms_preprocessed_withCtxt_flat.pickle'), 'rb') as f:
    X_flat, y_flat, T = pickle.load(f)

# Configurar device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Definir espacios de modelo base
mlp_base_space = {
    "model_class": MLPModel,
    "output_size": 1,
    "device": device,
    "num_epochs": 200,
    "es_patience": 10,
    "reg_type": None,
    "lambda_reg": None,
    "batch_size": 32,
    # Hiperparámetros a optimizar:
    "hidden_size": (int, 128, 512, 64),      # (tipo, low, high, step)
    "num_layers": 2,               # (tipo, low, high)
    "dropout": 0.4,            # (tipo, low, high)
    "lr": 1e-4
}

In [ ]:
run_study(
    X_flat, y_flat, T,
    model_space=mlp_base_space,
    num_trials=2,
    outer_folds=5,
    inner_folds=1,
    save_path=f"results/mlp_nested_cv"
)